In [1]:
!pip install -U gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.13.0
    Uninstalling gradio_client-1.13.0:
      Successfully uninstalled gradio_client-1.13.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.46.0
    Uninstalling gradio-5.46.0:
      Successfully uninstalled gradio-5.46.0


In [2]:
pip install gradio pandas numpy plotly statsmodels scikit-learn

In [3]:
!pip install openpyxl

In [4]:
!pip install gradio pandas openpyxl matplotlib --quiet

In [7]:
import gradio as gr
import pandas as pd
import tempfile
import os
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.seasonal import STL
import re
import warnings

warnings.filterwarnings("ignore")

# --- Global State ---
original_df = pd.DataFrame()
current_filtered_df = pd.DataFrame()
preview_visible = False
filtered_visible = False
tables_data = {}  # Dictionary to store table data
active_filter_values = {}  # column -> list of selected values (strings)
filter_columns_map = {}  # Map each filter dropdown index -> column name (kept in sync with current dataset)

# --- Constants ---
EXCLUDED_STAT_COLS = ['year', 'crop_year', 'id', 'code', 'index', 'no']
MAX_UNIQUE_FILTER = 500
MAX_FILTERS = 10
FILTER_ALL_TOKEN = "All"
temp_dir = tempfile.gettempdir()

# CSV/Excel settings (Excel-friendly)
CSV_ENCODING = "utf-8-sig"   # BOM so Excel detects UTF-8 automatically
CSV_DELIMITER = ";"          # French/European Excel uses semicolon; set to "," if desired

def write_csv(df: pd.DataFrame, path: str):
    df.to_csv(path, index=False, encoding=CSV_ENCODING, sep=CSV_DELIMITER)

# --- Helper Functions ---
def sanitize_token(s: str, max_len: int = 120) -> str:
    if s is None:
        return ""
    s = str(s)
    s = re.sub(r'\s+', '_', s.strip())
    s = re.sub(r'[^\w\-\+\=\.,]', '', s)
    return s[:max_len]

def coerce_datetime_columns(df: pd.DataFrame, threshold=0.7) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if pd.api.types.is_datetime64_any_dtype(out[c]):
            continue
        if out[c].dtype == object:
            parsed = pd.to_datetime(out[c], errors="coerce")
            if parsed.notna().mean() >= threshold:
                out[c] = parsed
    return out

def robust_to_numeric(series_like) -> pd.Series:
    s = pd.Series(series_like)
    if pd.api.types.is_numeric_dtype(s):
        return pd.to_numeric(s, errors='coerce')

    s = s.astype(str).str.strip()
    # Normalize non-breaking spaces and spaces
    s = s.str.replace('\u00A0', ' ', regex=False).str.replace('\xa0', ' ', regex=False)
    s = s.str.replace(r'\s+', '', regex=True)

    # Convert parentheses negatives: (123) -> -123
    s = s.str.replace(r'^\((.+)\)$', r'-\1', regex=True)

    # Detect decimal comma or values with comma but no dot
    dec_comma_pattern = r'^\s*[-+]?\d{1,3}(\.\d{3})*,\d+$'
    mask_decimal_comma = s.str.contains(dec_comma_pattern, regex=True) | (s.str.contains(',') & ~s.str.contains(r'\.'))

    s2 = s.copy()
    s2.loc[mask_decimal_comma] = (
        s2.loc[mask_decimal_comma]
          .str.replace('.', '', regex=False)
          .str.replace(',', '.', regex=False)
    )
    s2.loc[~mask_decimal_comma] = s2.loc[~mask_decimal_comma].str.replace(',', '', regex=False)
    s2 = s2.str.replace(r'[^0-9\.\-\+eE]', '', regex=True)
    return pd.to_numeric(s2, errors='coerce')

def safe_load_df(file):
    file_path = file.name if hasattr(file, "name") else file
    if file_path.endswith(".csv"):
        try:
            # Auto-detect delimiter robustly
            df = pd.read_csv(file_path, sep=None, engine='python', low_memory=False)
        except Exception:
            df = pd.read_csv(file_path, low_memory=False)
    elif file_path.endswith((".xlsx", ".xls")):
        df = pd.read_excel(file_path)
    else:
        raise ValueError("Unsupported file format. Please upload a CSV or Excel file.")
    return coerce_datetime_columns(df)

def visible_filter_options(col_name, unique_vals):
    choices = [FILTER_ALL_TOKEN] + sorted(unique_vals)
    if len(unique_vals) <= MAX_UNIQUE_FILTER:
        return gr.update(visible=True, choices=choices, value=None, label=str(col_name), interactive=True)
    return gr.update(visible=False, choices=[], value=None, label=str(col_name))

def get_filter_suffix_for_filename(max_len: int = 120) -> str:
    if not active_filter_values:
        return ""
    parts = []
    # Stable order by column name for reproducible filenames
    for col in sorted(active_filter_values.keys()):
        vals = active_filter_values[col]
        if not vals:
            continue
        if FILTER_ALL_TOKEN in vals:
            continue
        col_tok = sanitize_token(col, max_len=50)
        val_toks = [sanitize_token(v, max_len=50) for v in vals if v and v != FILTER_ALL_TOKEN]
        if not val_toks:
            continue
        parts.append(f"{col_tok}={'+'.join(val_toks)}")
    if not parts:
        return ""
    suffix = "__" + "__".join(parts)
    if len(suffix) > max_len:
        kept = []
        total = 2
        for p in parts:
            if total + len(p) + 2 > max_len:
                break
            kept.append(p)
            total += len(p) + 2
        suffix = "__" + "__".join(kept) if kept else ""
    return suffix

def attach_static_filter_cols(df: pd.DataFrame) -> pd.DataFrame:
    if df is None or df.empty:
        return df
    out = df.copy()
    for col, vals in active_filter_values.items():
        if FILTER_ALL_TOKEN in (vals or []):
            continue
        value_str = " | ".join([str(v) for v in (vals or [])]) if vals else ""
        if value_str and col not in out.columns:
            out[col] = value_str
    return out

def maybe_attach_static_cols(df: pd.DataFrame, meta_suffix: str) -> pd.DataFrame:
    if meta_suffix:
        return df
    return attach_static_filter_cols(df)

def make_temp_csv(df, prefix, meta_suffix: str = None):
    os.makedirs(temp_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    meta = sanitize_token(meta_suffix or "", max_len=200)
    filename = f"{prefix}{meta}_{timestamp}.csv" if meta else f"{prefix}_{timestamp}.csv"
    path = os.path.join(temp_dir, filename)
    try:
        write_csv(df, path)
        return path
    except Exception:
        filename = f"{prefix}{meta}_{timestamp}_{np.random.randint(1000,9999)}.csv" if meta else f"{prefix}_{timestamp}_{np.random.randint(1000,9999)}.csv"
        path = os.path.join(temp_dir, filename)
        write_csv(df, path)
        return path

def save_modified_table(table_name, modified_df):
    global temp_dir
    modified_dir = os.path.join(temp_dir, "modified_tables")
    os.makedirs(modified_dir, exist_ok=True)
    clean_name = re.sub(r'[^\w\-_]', '', str(table_name))
    suffix = sanitize_token(get_filter_suffix_for_filename(), max_len=200)
    csv_path = os.path.join(modified_dir, f"{clean_name}{suffix}_modified.csv")
    try:
        df_to_save = maybe_attach_static_cols(modified_df, suffix)
        write_csv(df_to_save, csv_path)
        return csv_path
    except Exception as e:
        print(f"Error saving modified table: {str(e)}")
        return None

def handle_table_modification(table_name, modified_df):
    global tables_data, current_filtered_df
    if modified_df is not None and not modified_df.empty:
        modified_df = coerce_datetime_columns(modified_df)
        current_filtered_df = modified_df.copy()
        tables_data[table_name] = current_filtered_df
        csv_path = save_modified_table(table_name, modified_df)
        meta = get_filter_suffix_for_filename()
        cur_df = maybe_attach_static_cols(current_filtered_df, meta)
        cur_path = make_temp_csv(cur_df, "current_data", meta)
        if csv_path:
            return gr.File(value=csv_path, visible=True), "Table updated successfully", gr.update(value=cur_path, visible=True)
    return gr.File(visible=False), "No changes applied", gr.update(visible=False)

# ---------------------------
# Math/Financial Functions
# ---------------------------

# Basic Arithmetic
def add_series(series, operand): return series + operand
def subtract_series(series, operand): return series - operand
def multiply_series(series, operand): return series * operand

def divide_series(series, operand, result_type='float'):
    if isinstance(operand, pd.Series):
        denom = operand.where(~np.isclose(operand, 0.0, atol=1e-12), np.nan)
    else:
        denom = np.nan if np.isclose(float(operand), 0.0, atol=1e-12) else operand
    result = series / denom
    rt = (result_type or 'float').lower()
    if rt == 'percent':
        result = result * 100.0
    elif rt == 'int':
        with np.errstate(invalid='ignore'):
            result = np.floor(result).astype('Int64')
    return result

# Statistical
def sum_series(series): return series.sum()

def crossec(series, method='mean'):
    method = (method or 'mean').lower()
    if method == 'mean': return series.mean()
    if method == 'sum': return series.sum()
    if method == 'max': return series.max()
    if method == 'min': return series.min()
    if method == 'median': return series.median()
    if method == 'std': return series.std()
    return series.mean()

def rollsum(series, period):
    period = int(period)
    if period <= 0: period = 1
    return series.rolling(window=period, min_periods=period).sum()

# Transformation
def accumulate(series, method='sum'):
    method = (method or 'sum').lower()
    if method == 'sum':
        return series.cumsum()
    elif method == 'avg':
        return series.expanding().mean()
    else:
        return series.cumsum()

def antilog(series): return np.exp(series)
def exp_series(series): return np.exp(series)

def log_series(series):
    s = series.copy()
    s = s.where(s > 0, np.nan)
    return np.log(s)

def log10_series(series):
    s = series.copy()
    s = s.where(s > 0, np.nan)
    return np.log10(s)

def power_series(series, exponent): return np.power(series, exponent)
def recip(series): return 1 / series.replace(0, np.nan)
def round_series(series, decimals=0): return series.round(int(decimals))

def sqrt_series(series):
    s = series.copy()
    s = s.where(s >= 0, np.nan)
    return np.sqrt(s)

# Time Series helpers
def percentage_change(series, period=1):
    return series.pct_change(periods=int(period)) * 100

def exponential_growth(series, period=1):
    # Same as percent change: rate of change over lag
    return series.pct_change(periods=int(period)) * 100

def compound_growth_rate(series, periods):
    periods = int(periods)
    s_clean = series.dropna()
    if len(s_clean) < 2 or periods <= 0:
        return np.nan
    start = s_clean.iloc[0]
    end = s_clean.iloc[-1]
    if start <= 0:
        return np.nan
    return ((end / start) ** (1 / periods) - 1) * 100

def difference(series, period=1):
    return series.diff(periods=int(period))

# Robust cleaner for resampling (handles duplicates safely)
def clean_time_series_for_resample(values, time_index, dup_agg='mean'):
    idx = pd.to_datetime(time_index, errors='coerce')
    vals = pd.to_numeric(pd.Series(values), errors="coerce")
    s = pd.Series(vals.values, index=idx)
    s = s[~s.index.isna()].sort_index()
    if s.index.has_duplicates:
        s = s.groupby(level=0).agg(dup_agg)
    return s.dropna()

def aggregate(series, time_index, frequency='M', method='sum', align_end=True):
    if time_index is None:
        raise ValueError("Please choose a time column (datetime) for aggregation.")
    s = clean_time_series_for_resample(series, time_index, dup_agg='mean')
    label = 'right' if align_end else 'left'
    closed = 'right' if align_end else 'left'
    res = s.resample(frequency, label=label, closed=closed)
    method = (method or 'sum').lower()
    if method == 'sum': out = res.sum()
    elif method == 'mean': out = res.mean()
    elif method == 'max': out = res.max()
    elif method == 'min': out = res.min()
    elif method == 'first': out = res.first()
    elif method == 'last': out = res.last()
    else: out = res.sum()
    return out

def disaggregate(series, time_index, frequency='D', method='linear', align_end=True):
    if time_index is None:
        raise ValueError("Please choose a time column (datetime) for disaggregation.")
    s = clean_time_series_for_resample(series, time_index, dup_agg='mean')
    label = 'right' if align_end else 'left'
    closed = 'right' if align_end else 'left'
    up = s.resample(frequency, label=label, closed=closed).asfreq()
    m = (method or 'linear').lower()
    if m == 'linear':
        out = up.interpolate(method='time')
    elif m == 'spline':
        try:
            out = up.interpolate(method='spline', order=3)
        except Exception:
            out = up.interpolate(method='time')
    elif m == 'ffill':
        out = up.ffill()
    else:
        out = up.interpolate(method='time')
    return out

# Conversion
def unit_conversion(series, multiplier): return series * float(multiplier)
def unit_multiplier(series, multiplier): return series * float(multiplier)
def convert_currency(series, rate): return series * float(rate)
def to_numeric_series(df, col): return robust_to_numeric(df[col])

# ---------------------------
# Deseasonalization helpers
# ---------------------------
def to_time_series(values, time_index):
    idx = pd.to_datetime(time_index, errors='coerce')
    vals = robust_to_numeric(values)
    mask = (~pd.isna(idx)) & (~pd.isna(vals.values))
    s = pd.Series(vals.values[mask], index=idx[mask]).sort_index()
    s = s.groupby(level=0).mean()
    return s

def deseasonalize_series(values, time_index, period, model='multiplicative'):
    if period is None or int(period) < 2:
        raise ValueError("Seasonal period must be an integer >= 2.")
    period = int(period)
    s = to_time_series(values, time_index)
    m = (str(model) or 'multiplicative').lower()
    if m.startswith('mult'):
        if (s <= 0).any():
            raise ValueError("Multiplicative deseasonalization requires positive values.")
        stl = STL(np.log(s), period=period, robust=True)
        res = stl.fit()
        deseason = np.exp(np.log(s) - res.seasonal)
        deseason = pd.Series(deseason, index=s.index)
    else:
        stl = STL(s, period=period, robust=True)
        res = stl.fit()
        deseason = s - res.seasonal
    return deseason

def deseasonalize_with_cycle(values, time_index, period, model='multiplicative'):
    if period is None or int(period) < 2:
        raise ValueError("Seasonal period must be an integer >= 2.")
    period = int(period)
    s = to_time_series(values, time_index)
    m = (str(model) or 'multiplicative').lower()
    if m.startswith('mult'):
        if (s <= 0).any():
            raise ValueError("Multiplicative deseasonalization requires strictly positive values.")
        stl = STL(np.log(s), period=period, robust=True)
        res = stl.fit()
        seasonal_series = np.exp(res.seasonal)
        ds = s / seasonal_series
        kind = 'factor'
    else:
        stl = STL(s, period=period, robust=True)
        res = stl.fit()
        seasonal_series = res.seasonal
        ds = s - seasonal_series
        kind = 'additive'
    n = len(s)
    positions = np.arange(n) % period
    cycle = np.zeros(period, dtype=float)
    for k in range(period):
        vals = np.asarray(seasonal_series)[positions == k]
        cycle[k] = float(np.mean(vals)) if len(vals) else (1.0 if kind == 'factor' else 0.0)
    last_pos = (n - 1) % period
    return ds, s, cycle, kind, last_pos

def reseasonalize_from_cycle(ds_forecast, start_pos, cycle, kind):
    n = len(ds_forecast)
    period = len(cycle)
    out = np.empty(n, dtype=float)
    for i in range(n):
        cyc_val = cycle[(start_pos + 1 + i) % period]
        out[i] = ds_forecast[i] + cyc_val if kind == 'additive' else ds_forecast[i] * cyc_val
    return out

def make_ds_preview_fig(index, original, deseasonalized):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=index, y=np.asarray(original), mode='lines+markers', name='Original Time Series',
                             line=dict(color='#1f77b4', width=2), marker=dict(size=4, opacity=0.8)))
    fig.add_trace(go.Scatter(x=index, y=np.asarray(deseasonalized), mode='lines+markers', name='Deseasonalized Time Series',
                             line=dict(color='#2ca02c', width=2.5), marker=dict(size=4, opacity=0.8)))
    fig.update_layout(
        title=dict(text='Original vs. Deseasonalized Time Series', font=dict(size=18, color='#333')),
        xaxis_title='Date', yaxis_title='Value', hovermode='x unified', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        template='plotly_white', margin=dict(l=40, r=40, t=60, b=40),
        xaxis=dict(showgrid=True, gridcolor='#e0e0e0'), yaxis=dict(showgrid=True, gridcolor='#e0e0e0')
    )
    return fig

def make_empty_note_fig(note_text):
    fig = go.Figure()
    fig.add_annotation(text=note_text, xref="paper", yref="paper", x=0.5, y=0.5, showarrow=False, font=dict(size=14, color='#666'))
    fig.update_layout(xaxis=dict(visible=False), yaxis=dict(visible=False), title=dict(text="Deseasonalization Preview", font=dict(size=16)))
    return fig

def detect_datetime_cols(df):
    return [c for c in df.columns if pd.api.types.is_datetime64_any_dtype(df[c])]

def get_series_with_time(df, column_name, time_column=None):
    # Return series and associated time index if provided
    if time_column and time_column in df.columns:
        idx = pd.to_datetime(df[time_column], errors='coerce')
    else:
        tcols = detect_datetime_cols(df)
        idx = pd.to_datetime(df[tcols[0]], errors='coerce') if tcols else None

    s = robust_to_numeric(df[column_name])
    if idx is not None:
        tmp = pd.DataFrame({"_val": s, "_t": idx}).dropna(subset=["_val", "_t"]).sort_values("_t")
        return tmp["_val"].reset_index(drop=True), tmp["_t"].reset_index(drop=True)
    else:
        # No time column – return None for time to avoid fake dates downstream
        return s.reset_index(drop=True), None

def format_date_series_for_output(dt_index_like):
    try:
        if isinstance(dt_index_like, pd.DatetimeIndex):
            return dt_index_like.strftime('%Y-%m-%d').tolist()
        if isinstance(dt_index_like, pd.PeriodIndex):
            return dt_index_like.to_timestamp().strftime('%Y-%m-%d').tolist()
        s = pd.Series(dt_index_like)
        dt = pd.to_datetime(s, errors='coerce')
        if dt.dropna().empty:
            return s.astype(str).tolist()
        return dt.dt.strftime('%Y-%m-%d').tolist()
    except Exception:
        try:
            return pd.Series(dt_index_like).astype(str).tolist()
        except Exception:
            return dt_index_like

# ---------------------------
# Math Application
# ---------------------------
def apply_math_function(
    selected_function, column_name,
    operand_source, operand_value, operand_column,
    numeric_param, method_param, frequency_param,
    result_type, time_column, align_end,
    ds_use_in_math, ds_math_period, ds_math_model,
    add_to_data, result_col_name, show_overlay
):
    global current_filtered_df, tables_data

    def _dropdown_updates_for_current_df(new_col_preselect=None):
        numeric_cols = current_filtered_df.select_dtypes(include=[np.number]).columns.tolist()
        time_cols = detect_datetime_cols(current_filtered_df)
        arima_col_upd = gr.update(choices=numeric_cols, value=(new_col_preselect if (new_col_preselect in numeric_cols) else (numeric_cols[0] if numeric_cols else None)))
        arima_time_upd = gr.update(choices=time_cols, value=(time_cols[0] if time_cols else None))
        math_col_upd = gr.update(choices=numeric_cols, value=(new_col_preselect if (new_col_preselect in numeric_cols) else (numeric_cols[0] if numeric_cols else None)))
        table_sel_upd = gr.update(choices=list(tables_data.keys()), value=("Transformed Table" if "Transformed Table" in tables_data else (list(tables_data.keys())[0] if tables_data else None)))
        table_prev_df = tables_data.get("Transformed Table", current_filtered_df)
        return arima_col_upd, arima_time_upd, math_col_upd, table_sel_upd, table_prev_df

    if current_filtered_df.empty or column_name not in current_filtered_df.columns:
        ar_col, ar_time, m_col, t_sel, t_prev = _dropdown_updates_for_current_df()
        return None, "Error: No data or column not found", gr.update(visible=False), gr.update(visible=False), go.Figure(), ar_col, ar_time, m_col, t_sel, t_prev

    fn = (selected_function or "").upper()
    user_col = (result_col_name or "").strip()
    final_col_name = user_col if user_col else ("Deseasonalized" if fn == "DESEASONALIZE" else f"{fn}_{column_name}")

    base_series, base_time = get_series_with_time(current_filtered_df, column_name, time_column)
    series = pd.Series(base_series)

    have_time = (time_column is not None and time_column in current_filtered_df.columns and base_time is not None)
    orig_ts = clean_time_series_for_resample(series, base_time, dup_agg='mean') if have_time else None

    # Use deseasonalized input optionally (except when running the DESEASONALIZE function itself)
    use_ds = bool(ds_use_in_math) and (fn != "DESEASONALIZE")
    in_ts = orig_ts
    if use_ds:
        if not have_time:
            ar_col, ar_time, m_col, t_sel, t_prev = _dropdown_updates_for_current_df()
            return None, "Error: Deseasonalized input requires a Time Column.", gr.update(visible=False), gr.update(visible=False), go.Figure(), ar_col, ar_time, m_col, t_sel, t_prev
        if ds_math_period is None or int(ds_math_period) < 2:
            ar_col, ar_time, m_col, t_sel, t_prev = _dropdown_updates_for_current_df()
            return None, "Error: DS Seasonal Period must be >= 2.", gr.update(visible=False), gr.update(visible=False), go.Figure(), ar_col, ar_time, m_col, t_sel, t_prev
        ds_model = ds_math_model or 'multiplicative'
        # Compute deseasonalized time series based on original time series
        try:
            ds_ts = deseasonalize_series(orig_ts.values, orig_ts.index, int(ds_math_period), model=ds_model)
        except Exception as de:
            ar_col, ar_time, m_col, t_sel, t_prev = _dropdown_updates_for_current_df()
            return None, f"Error applying deseasonalized input: {de}", gr.update(visible=False), gr.update(visible=False), go.Figure(), ar_col, ar_time, m_col, t_sel, t_prev
        in_ts = ds_ts
        series = ds_ts.copy()  # for non-time functions

    # Clean, time-ordered series for time-dependent functions
    time_funcs = {"ROLLSUM", "%CHANGE", "%EXPGROWTH", "CAGR", "DIFF", "ACCUMULATE"}
    ts = in_ts if (fn in time_funcs) else None

    def get_operand():
        if fn in ["ADD", "SUBTRACT", "MULTIPLY", "DIVIDE"]:
            src = (operand_source or 'value').lower()
            if src == 'column':
                if not operand_column or operand_column not in current_filtered_df.columns:
                    raise ValueError("Choose a valid operand column.")
                op_vals, op_time = get_series_with_time(current_filtered_df, operand_column, time_column if have_time else None)
                if have_time:
                    op_ts = clean_time_series_for_resample(op_vals, op_time, dup_agg='mean')
                    # Align to the base input (deseasonalized if enabled)
                    base_align = in_ts if in_ts is not None else orig_ts
                    return op_ts.reindex(base_align.index)
                else:
                    return pd.Series(op_vals).reset_index(drop=True).reindex_like(series)
            else:
                if operand_value is None:
                    raise ValueError("Provide a numeric value for the operation.")
                return float(operand_value)
        return None

    try:
        result = None

        if fn in ["ACCUMULATE", "ANTILOG", "EXP", "LOG", "LOG10", "RECIP", "SQRT", "SUM", "CROSSSEC"]:
            if fn == "ACCUMULATE":
                if ts is None:
                    raise ValueError("ACCUMULATE requires a Time Column.")
                result = accumulate(ts, method_param or 'sum')
            elif fn == "ANTILOG": result = antilog(series)
            elif fn == "EXP": result = exp_series(series)
            elif fn == "LOG": result = log_series(series)
            elif fn == "LOG10": result = log10_series(series)
            elif fn == "RECIP": result = recip(series)
            elif fn == "SQRT": result = sqrt_series(series)
            elif fn == "SUM": result = sum_series(series)
            elif fn == "CROSSSEC": result = crossec(series, method_param or 'mean')

        elif fn in ["ADD", "SUBTRACT", "MULTIPLY", "DIVIDE"]:
            operand = get_operand()
            base_in = in_ts if have_time else series
            if isinstance(operand, pd.Series):
                if fn == "ADD": result = add_series(base_in, operand)
                elif fn == "SUBTRACT": result = subtract_series(base_in, operand)
                elif fn == "MULTIPLY": result = multiply_series(base_in, operand)
                elif fn == "DIVIDE": result = divide_series(base_in, operand, result_type)
            else:
                if fn == "ADD": result = add_series(base_in, float(operand))
                elif fn == "SUBTRACT": result = subtract_series(base_in, float(operand))
                elif fn == "MULTIPLY": result = multiply_series(base_in, float(operand))
                elif fn == "DIVIDE": result = divide_series(base_in, float(operand), result_type)

        elif fn == "POWER":
            if numeric_param is None:
                raise ValueError("Exponent is required for POWER function.")
            result = power_series(series, float(numeric_param))

        elif fn == "ROLLSUM":
            if numeric_param is None:
                raise ValueError("Window size is required for ROLLSUM.")
            if ts is None:
                raise ValueError("ROLLSUM requires a Time Column.")
            result = rollsum(ts, int(numeric_param))

        elif fn == "ROUND":
            result = round_series(series, int(numeric_param if numeric_param is not None else 0))

        elif fn == "%CHANGE":
            if numeric_param is None:
                raise ValueError("Period is required for %CHANGE function.")
            if ts is None:
                raise ValueError("%CHANGE requires a Time Column.")
            result = percentage_change(ts, int(numeric_param))

        elif fn == "%EXPGROWTH":
            if numeric_param is None:
                raise ValueError("Period is required for %EXPGROWTH function.")
            if ts is None:
                raise ValueError("%EXPGROWTH requires a Time Column.")
            result = exponential_growth(ts, int(numeric_param))

        elif fn == "CAGR":
            if numeric_param is None:
                raise ValueError("Compounding periods are required for CAGR function.")
            if ts is None:
                raise ValueError("CAGR requires a Time Column.")
            result = compound_growth_rate(ts, int(numeric_param))

        elif fn == "DIFF":
            if numeric_param is None:
                raise ValueError("Period is required for DIFF function.")
            if ts is None:
                raise ValueError("DIFF requires a Time Column.")
            result = difference(ts, int(numeric_param))

        elif fn in ["UNIT_CONVERSION", "UNIT_MULTIPLIER", "CONVERTCUR"]:
            if numeric_param is None:
                raise ValueError(f"Multiplier/rate is required for {fn} function.")
            factor = float(numeric_param)
            if fn == "UNIT_CONVERSION": result = unit_conversion(series, factor)
            elif fn == "UNIT_MULTIPLIER": result = unit_multiplier(series, factor)
            else: result = convert_currency(series, factor)

        elif fn == "AGGREGATE":
            if time_column is None or time_column not in current_filtered_df.columns:
                raise ValueError("Please choose a Time Column (datetime) for AGGREGATE.")
            freq = frequency_param or 'M'
            if use_ds and in_ts is not None:
                result = aggregate(in_ts.values, in_ts.index, frequency=freq, method=method_param or 'sum', align_end=bool(align_end))
            else:
                result = aggregate(to_numeric_series(current_filtered_df, column_name),
                                   current_filtered_df[time_column],
                                   frequency=freq, method=method_param or 'sum', align_end=bool(align_end))
        elif fn == "DISAGGREGATE":
            if time_column is None or time_column not in current_filtered_df.columns:
                raise ValueError("Please choose a Time Column (datetime) for DISAGGREGATE.")
            freq = frequency_param or 'D'
            if use_ds and in_ts is not None:
                result = disaggregate(in_ts.values, in_ts.index, frequency=freq, method=method_param or 'linear', align_end=bool(align_end))
            else:
                result = disaggregate(to_numeric_series(current_filtered_df, column_name),
                                      current_filtered_df[time_column],
                                      frequency=freq, method=method_param or 'linear', align_end=bool(align_end))
        elif fn == "DESEASONALIZE":
            if time_column is None or time_column not in current_filtered_df.columns:
                raise ValueError("Please choose a Time Column for Deseasonalization.")
            if numeric_param is None:
                raise ValueError("Seasonal period is required for Deseasonalization.")
            model = method_param or 'multiplicative'
            result = deseasonalize_series(to_numeric_series(current_filtered_df, column_name),
                                          current_filtered_df[time_column], int(numeric_param), model=model)
        else:
            ar_col, ar_time, m_col, t_sel, t_prev = _dropdown_updates_for_current_df()
            return None, "Error: Function not implemented", gr.update(visible=False), gr.update(visible=False), go.Figure(), ar_col, ar_time, m_col, t_sel, t_prev

        scalar_output = np.isscalar(result)

        # Build output df
        if scalar_output:
            out_df = pd.DataFrame({"Function": [fn], "Column": [column_name], "Result": [result]})
        else:
            if isinstance(result, pd.Series) and isinstance(result.index, pd.DatetimeIndex):
                ser = result
                out_df = pd.DataFrame({"Date": format_date_series_for_output(ser.index), final_col_name: ser.values})
            else:
                ser = pd.Series(result).reset_index(drop=True)
                if base_time is not None and len(ser) == len(pd.Series(base_series)):
                    out_df = pd.DataFrame({final_col_name: ser.values})
                else:
                    out_df = pd.DataFrame({final_col_name: ser.values})

        # Save download
        meta = get_filter_suffix_for_filename()
        out_save = maybe_attach_static_cols(out_df, meta)
        math_path = make_temp_csv(out_save, f"math_{fn.lower()}", meta)

        cur_download_update = gr.update(visible=False)
        new_col_name = None
        status_msg = f"Success: {fn}" + (" (using deseasonalized input)" if use_ds else "")

        if not scalar_output and add_to_data:
            new_col_name = final_col_name
            if isinstance(result, pd.Series) and isinstance(result.index, pd.DatetimeIndex) and fn in ["AGGREGATE", "DISAGGREGATE"]:
                time_name = time_column or "Period"
                transformed_df = out_df.rename(columns={"Date": time_name})
                if "Date" in transformed_df.columns:
                    transformed_df[time_name] = pd.to_datetime(transformed_df[time_name], errors="coerce")
                    transformed_df = transformed_df.drop(columns=["Date"])
                current_filtered_df = transformed_df.copy()
                tables_data["Transformed Table"] = current_filtered_df.copy()
                status_msg = f"Success: {fn}" + (" (using deseasonalized input)" if use_ds else "") + f" | Created 'Transformed Table' with [{time_name}, {new_col_name}] and set it as current data."
            else:
                if isinstance(result, pd.Series) and isinstance(result.index, pd.DatetimeIndex) and have_time and time_column in current_filtered_df.columns:
                    temp = pd.DataFrame({time_column: result.index, new_col_name: result.values})
                    temp[time_column] = pd.to_datetime(temp[time_column], errors="coerce")
                    current_filtered_df[time_column] = pd.to_datetime(current_filtered_df[time_column], errors="coerce")
                    current_filtered_df = current_filtered_df.merge(temp[[time_column, new_col_name]], on=time_column, how="left")
                    status_msg = f"Success: {fn}" + (" (using deseasonalized input)" if use_ds else "") + f" | Added column '{new_col_name}' aligned by time."
                elif len(pd.Series(result)) != len(current_filtered_df):
                    status_msg = f"Success: {fn}" + (" (using deseasonalized input)" if use_ds else "") + " | Result length differs from data; not added."
                else:
                    current_filtered_df[new_col_name] = pd.Series(result).values
                    status_msg = f"Success: {fn}" + (" (using deseasonalized input)" if use_ds else "") + f" | Added column '{new_col_name}' to current data."

                tables_data["Transformed Table"] = current_filtered_df.copy()

            cur_df = maybe_attach_static_cols(current_filtered_df, meta)
            cur_path = make_temp_csv(cur_df, "current_data", meta)
            cur_download_update = gr.update(value=cur_path, visible=True)

        # Visualization (two separate charts when overlay is enabled)
        result_is_percent = (fn in ["%CHANGE", "%EXPGROWTH"]) or (fn == "DIVIDE" and (result_type or "").lower() == "percent")

        if scalar_output:
            fig = go.Figure()
            fig.add_trace(go.Indicator(mode="number", value=result, title=dict(text=fn, font=dict(size=18))))
            fig.update_layout(title=dict(text=f"Result of {fn}", font=dict(size=20, color='#333')), template='plotly_white')
        else:
            if isinstance(result, pd.Series) and isinstance(result.index, pd.DatetimeIndex):
                x_trans = result.index
                y_trans = result.values
            else:
                ser = pd.Series(result).reset_index(drop=True)
                if have_time and in_ts is not None:
                    x_trans = np.arange(len(ser)) if len(ser) != len(in_ts) else in_ts.index
                else:
                    x_trans = np.arange(len(ser))
                y_trans = ser.values

            # Original series for overlay (always original, not deseasonalized)
            if have_time and orig_ts is not None:
                x_orig = orig_ts.index
                y_original = orig_ts.values
            else:
                x_orig = np.arange(len(series))
                y_original = pd.Series(base_series).values

            if show_overlay:
                fig = make_subplots(
                    rows=2, cols=1, shared_xaxes=False, vertical_spacing=0.08,
                    subplot_titles=("Original Series", f"{final_col_name}" + (" (%)" if result_is_percent else ""))
                )
                fig.add_trace(go.Scatter(x=x_orig, y=y_original, mode='lines+markers', name='Original Series', line=dict(color='#1f77b4', width=2), marker=dict(size=4, opacity=0.8)), row=1, col=1)
                fig.add_trace(go.Scatter(x=x_trans, y=y_trans, mode='lines+markers', name=final_col_name, line=dict(color='#d62728', width=2), marker=dict(size=4, opacity=0.8)), row=2, col=1)
                fig.update_yaxes(title_text="Value", row=1, col=1)
                fig.update_yaxes(title_text=("Percent (%)" if result_is_percent else "Value"), row=2, col=1)
                fig.update_xaxes(title_text="Date/Index", title_standoff=8, row=1, col=1)
                fig.update_xaxes(title_text="Date/Index", title_standoff=8, row=2, col=1)
                fig.update_layout(title=dict(text=f"Original (top) and {final_col_name} (bottom)", font=dict(size=18, color='#333')), hovermode='x unified', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1), template='plotly_white')
            else:
                fig = go.Figure()
                fig.add_trace(go.Scatter(x=x_trans, y=y_trans, mode='lines+markers', name=final_col_name, line=dict(color='#d62728', width=2), marker=dict(size=4, opacity=0.8)))
                y_title = 'Percent (%)' if result_is_percent else 'Value'
                fig.update_layout(title=dict(text=f"Transformed Series using {fn}", font=dict(size=18, color='#333')), xaxis_title="Date/Index", yaxis_title=y_title, hovermode='x unified', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1), template='plotly_white')

        ar_col_upd, ar_time_upd, math_col_upd, table_sel_upd, table_prev_df = _dropdown_updates_for_current_df(new_col_preselect=new_col_name)

        return out_df, status_msg, gr.update(value=math_path, visible=True), cur_download_update, fig, \
               ar_col_upd, ar_time_upd, math_col_upd, table_sel_upd, table_prev_df

    except Exception as e:
        print(f"Math Error: {e}")
        ar_col_upd, ar_time_upd, math_col_upd, table_sel_upd, table_prev_df = _dropdown_updates_for_current_df()
        return None, f"Error in {selected_function}: {str(e)}", gr.update(visible=False), gr.update(visible=False), go.Figure(), \
               ar_col_upd, ar_time_upd, math_col_upd, table_sel_upd, table_prev_df

# ---------------------------
# ARIMA helpers (updated DS options)
# ---------------------------
def calculate_smape(actual, forecast):
    actual = np.array(actual)
    forecast = np.array(forecast)
    denom = np.abs(actual) + np.abs(forecast)
    denom = np.where(denom == 0, 1e-10, denom)
    smape_values = 2.0 * np.abs(forecast - actual) / denom
    smape_values = smape_values[np.isfinite(smape_values)]
    if len(smape_values) == 0:
        return 0.0
    return np.mean(smape_values) * 100

def run_arima(column_name, periods=5, p=1, d=1, q=1, time_col=None, freq_choice='infer',
              overlay_deseasonalize=True, ds_period=12, ds_model='multiplicative',
              use_ds_for_arima=False, reseasonalize_output=True):
    global current_filtered_df
    if current_filtered_df.empty or column_name not in current_filtered_df.columns:
        return None, None, "Please load data and select a valid numeric column.", make_empty_note_fig("No data"), gr.update(visible=False), gr.update(visible=False)
    if not time_col or time_col not in current_filtered_df.columns or not pd.api.types.is_datetime64_any_dtype(current_filtered_df[time_col]):
        return None, None, "Please select a time column (datetime) for ARIMA.", make_empty_note_fig("Select time column"), gr.update(visible=False), gr.update(visible=False)
    try:
        df2 = current_filtered_df[[time_col, column_name]].copy()
        df2[time_col] = pd.to_datetime(df2[time_col], errors='coerce')
        df2[column_name] = robust_to_numeric(df2[column_name])
        df2 = df2.dropna(subset=[time_col, column_name]).sort_values(time_col)
        s = df2.groupby(time_col)[column_name].mean()
        if len(s) < 10:
            raise ValueError("Insufficient data points for ARIMA (min: 10 after cleaning).")

        # Frequency inference
        if freq_choice and str(freq_choice).lower() != "infer":
            freq_to_use = str(freq_choice)
        else:
            try:
                freq_to_use = pd.infer_freq(s.index)
            except Exception:
                freq_to_use = None

        overlay_msg = ""
        ds_preview_fig = make_empty_note_fig("Enable overlay or DS-input to preview deseasonalization")

        ds_for_overlay = None
        ds_series_download_update = gr.update(visible=False)
        # Overlay DS preview (separate from using DS for ARIMA)
        if overlay_deseasonalize:
            try:
                ds_for_overlay = deseasonalize_series(s.values, s.index, int(ds_period or 12), model=ds_model)
                ds_preview_fig = make_ds_preview_fig(ds_for_overlay.index, s.loc[ds_for_overlay.index].values, ds_for_overlay.values)
                ds_df = pd.DataFrame({"Period": format_date_series_for_output(ds_for_overlay.index), "Deseasonalized": ds_for_overlay.values})
                meta = get_filter_suffix_for_filename()
                ds_df_save = maybe_attach_static_cols(ds_df, meta)
                ds_path = make_temp_csv(ds_df_save, "deseasonalized_series", meta)
                ds_series_download_update = gr.update(value=ds_path, visible=True)
            except Exception as de:
                overlay_msg = f" | Deseasonalization overlay failed: {de}"
                ds_preview_fig = make_empty_note_fig(f"Deseasonalization failed: {de}")
                ds_series_download_update = gr.update(visible=False)

        if use_ds_for_arima:
            effective_model = ds_model
            try:
                ds_series, s_aligned, cycle, kind, last_pos = deseasonalize_with_cycle(
                    s.values, s.index, int(ds_period or 12), model=ds_model
                )
            except Exception as e:
                ds_series, s_aligned, cycle, kind, last_pos = deseasonalize_with_cycle(
                    s.values, s.index, int(ds_period or 12), model='additive'
                )
                effective_model = 'additive'
                overlay_msg += f" | Multiplicative DS not possible: {e}. Fallback to additive."

            if ds_for_overlay is None:
                ds_preview_fig = make_ds_preview_fig(s_aligned.index, s_aligned.values, ds_series.values)
                ds_df2 = pd.DataFrame({"Period": format_date_series_for_output(ds_series.index), "Deseasonalized": ds_series.values})
                meta = get_filter_suffix_for_filename()
                ds_series_download_update = gr.update(value=make_temp_csv(maybe_attach_static_cols(ds_df2, meta), "deseasonalized_series", meta), visible=True)

            ds_vals = ds_series.values
            orig_vals = s_aligned.values
            n = len(ds_vals)
            split_point = max(1, int(n * 0.8))
            train_ds, test_ds = ds_vals[:split_point], ds_vals[split_point:]
            test_orig = orig_vals[split_point:]

            model_v = ARIMA(train_ds, order=(int(p), int(d), int(q))).fit()
            fc_val_ds = model_v.forecast(steps=len(test_ds))

            # Metrics: re-seasonalize if requested; else evaluate in DS domain
            if reseasonalize_output:
                val_pred = reseasonalize_from_cycle(
                    np.asarray(fc_val_ds), start_pos=(split_point - 1) % int(ds_period or 12),
                    cycle=cycle, kind=('additive' if effective_model == 'additive' else 'factor')
                )
                mae = mean_absolute_error(test_orig, val_pred) if len(test_orig) > 0 else np.nan
                rmse = np.sqrt(mean_squared_error(test_orig, val_pred)) if len(test_orig) > 0 else np.nan
                smape = calculate_smape(test_orig, val_pred) if len(test_orig) > 0 else np.nan
                metrics_text = (
                    f"Validation (20% holdout) — ARIMA on deseasonalized ({effective_model}); output re-seasonalized:\n"
                    f"Model: ARIMA({int(p)},{int(d)},{int(q)})\n"
                    f"MAE: {mae:.4f}\nRMSE: {rmse:.4f}\nsMAPE: {smape:.2f}%{overlay_msg}"
                )
            else:
                mae = mean_absolute_error(test_ds, fc_val_ds) if len(test_ds) > 0 else np.nan
                rmse = np.sqrt(mean_squared_error(test_ds, fc_val_ds)) if len(test_ds) > 0 else np.nan
                smape = calculate_smape(test_ds, fc_val_ds) if len(test_ds) > 0 else np.nan
                metrics_text = (
                    f"Validation (20% holdout) — ARIMA on deseasonalized ({effective_model}); output in DS units:\n"
                    f"Model: ARIMA({int(p)},{int(d)},{int(q)})\n"
                    f"MAE (DS): {mae:.4f}\nRMSE (DS): {rmse:.4f}\nsMAPE (DS): {smape:.2f}%{overlay_msg}"
                )

            model_full = ARIMA(ds_vals, order=(int(p), int(d), int(q))).fit()
            fc_ds_future = model_full.forecast(steps=int(periods))

            last_date = s.index[-1]
            if freq_to_use:
                offset = pd.tseries.frequencies.to_offset(freq_to_use)
                future_index = pd.date_range(start=last_date + offset, periods=int(periods), freq=offset)
            else:
                diffs = s.index.to_series().diff().dropna()
                delta_td = pd.to_timedelta(diffs.mode().iloc[0]) if len(diffs) > 0 else pd.Timedelta(days=1)
                future_index = pd.date_range(start=last_date + delta_td, periods=int(periods), freq=delta_td)

            if reseasonalize_output:
                reseason_future = reseasonalize_from_cycle(
                    np.asarray(fc_ds_future),
                    start_pos=last_pos, cycle=cycle,
                    kind=('additive' if effective_model == 'additive' else 'factor')
                )
                forecast_df = pd.DataFrame({'Period': future_index, 'Forecast Value': reseason_future})
            else:
                forecast_df = pd.DataFrame({'Period': future_index, 'Forecast Value (DS)': np.asarray(fc_ds_future)})

            history_df = pd.DataFrame({'Period': s.index, 'Value': s.values})

            fig = go.Figure()
            fig.add_trace(go.Scatter(x=history_df['Period'], y=history_df['Value'],
                                     mode='lines+markers', name='Original Time Series',
                                     line=dict(color='#1f77b4', width=2), marker=dict(color='#1f77b4', size=4, opacity=0.8)))
            ds_plot_series = ds_for_overlay if ds_for_overlay is not None else ds_series
            fig.add_trace(go.Scatter(x=ds_plot_series.index, y=ds_plot_series.values,
                                     mode='lines+markers', name='Deseasonalized Time Series',
                                     line=dict(color='#2ca02c', width=2.5), marker=dict(size=4, opacity=0.8)))
            if reseasonalize_output:
                fig.add_trace(go.Scatter(x=forecast_df['Period'], y=forecast_df['Forecast Value'],
                                         mode='lines+markers', name='Forecast (reseasonalized)',
                                         line=dict(color='#d62728', dash='dot', width=2), marker=dict(color='#d62728', size=4, opacity=0.8)))
                y_title = 'Value'
            else:
                fig.add_trace(go.Scatter(x=forecast_df['Period'], y=forecast_df['Forecast Value (DS)'],
                                         mode='lines+markers', name='Forecast (DS)',
                                         line=dict(color='#d62728', dash='dot', width=2), marker=dict(color='#d62728', size=4, opacity=0.8)))
                y_title = 'Value (DS)'

            fig.update_layout(
                title=dict(text=f'ARIMA — DS input={bool(use_ds_for_arima)}, Reseason={bool(reseasonalize_output)} — ARIMA({int(p)},{int(d)},{int(q)}) for {column_name}', font=dict(size=18, color='#333')),
                xaxis_title='Date', yaxis_title=y_title, hovermode='x unified', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
                template='plotly_white', margin=dict(l=40, r=40, t=60, b=40),
                xaxis=dict(showgrid=True, gridcolor='#e0e0e0'), yaxis=dict(showgrid=True, gridcolor='#e0e0e0')
            )

            forecast_df_display = forecast_df.copy()
            forecast_df_display['Period'] = format_date_series_for_output(forecast_df_display['Period'])
            meta = get_filter_suffix_for_filename()
            forecast_df_display = maybe_attach_static_cols(forecast_df_display, meta)
            fc_path = make_temp_csv(forecast_df_display, "arima_forecast", meta)
            return forecast_df_display, fig, metrics_text, ds_preview_fig, gr.update(value=fc_path, visible=True), ds_series_download_update

        # Standard ARIMA on raw series
        values = s.values
        split_point = max(1, int(len(values) * 0.8))
        model_val = ARIMA(train := values[:split_point], order=(int(p), int(d), int(q))).fit()
        forecast_val = model_val.forecast(steps=len(values[split_point:]))
        mae = mean_absolute_error(values[split_point:], forecast_val) if len(values[split_point:]) > 0 else np.nan
        rmse = np.sqrt(mean_squared_error(values[split_point:], forecast_val)) if len(values[split_point:]) > 0 else np.nan
        smape = calculate_smape(values[split_point:], forecast_val) if len(values[split_point:]) > 0 else np.nan
        metrics_text = (
            f"Validation Metrics (20% holdout):\n"
            f"Model: ARIMA({int(p)},{int(d)},{int(q)})\n"
            f"MAE: {mae:.4f}\nRMSE: {rmse:.4f}\nsMAPE: {smape:.2f}%"
        )

        model = ARIMA(values, order=(int(p), int(d), int(q))).fit()
        fc_values = model.forecast(steps=int(periods))

        if freq_to_use:
            offset = pd.tseries.frequencies.to_offset(freq_to_use)
            future_index = pd.date_range(start=s.index[-1] + offset, periods=int(periods), freq=offset)
        else:
            diffs = s.index.to_series().diff().dropna()
            delta_td = pd.to_timedelta(diffs.mode().iloc[0]) if len(diffs) > 0 else pd.Timedelta(days=1)
            future_index = pd.date_range(start=s.index[-1] + delta_td, periods=int(periods), freq=delta_td)

        history_df = pd.DataFrame({'Period': s.index, 'Value': s.values})
        forecast_df = pd.DataFrame({'Period': future_index, 'Forecast Value': np.asarray(fc_values)})

        fig = go.Figure()
        fig.add_trace(go.Scatter(x=history_df['Period'], y=history_df['Value'],
                                 mode='lines+markers', name='Original Time Series',
                                 line=dict(color='#1f77b4', width=2), marker=dict(color='#1f77b4', size=4, opacity=0.8)))
        fig.add_trace(go.Scatter(x=forecast_df['Period'], y=forecast_df['Forecast Value'],
                                 mode='lines+markers', name='Forecast',
                                 line=dict(color='#d62728', dash='dot', width=2), marker=dict(color='#d62728', size=4, opacity=0.8)))
        fig.update_layout(
            title=dict(text=f'ARIMA Forecast — ARIMA({int(p)},{int(d)},{int(q)}) for {column_name}', font=dict(size=18, color='#333')),
            xaxis_title='Date', yaxis_title='Value', hovermode='x unified', legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
            template='plotly_white', margin=dict(l=40, r=40, t=60, b=40),
            xaxis=dict(showgrid=True, gridcolor='#e0e0e0'), yaxis=dict(showgrid=True, gridcolor='#e0e0e0')
        )

        forecast_df_display = forecast_df.copy()
        forecast_df_display['Period'] = format_date_series_for_output(forecast_df_display['Period'])
        meta = get_filter_suffix_for_filename()
        forecast_df_display = maybe_attach_static_cols(forecast_df_display, meta)

        fc_path = make_temp_csv(forecast_df_display, "arima_forecast", meta)
        return forecast_df_display, fig, metrics_text, ds_preview_fig, gr.update(value=fc_path, visible=True), gr.update(visible=False)

    except Exception as e:
        print(f"ARIMA Error: {e}")
        return None, None, str(e), make_empty_note_fig(f"Error: {e}"), gr.update(visible=False), gr.update(visible=False)

# ---------------------------
# Dynamic UI Helpers
# ---------------------------
def math_function_config(func_name):
    f = (func_name or "").upper()
    show_operand = f in ["ADD", "SUBTRACT", "MULTIPLY", "DIVIDE"]
    show_numeric = f in ["POWER", "ROLLSUM", "ROUND", "%CHANGE", "%EXPGROWTH", "CAGR", "DIFF", "UNIT_CONVERSION", "UNIT_MULTIPLIER", "CONVERTCUR", "DESEASONALIZE"]
    numeric_label_map = {
        "POWER": "Exponent",
        "ROLLSUM": "Window (period)",
        "ROUND": "Decimals",
        "%CHANGE": "Period",
        "%EXPGROWTH": "Period",
        "CAGR": "Compounding periods",
        "DIFF": "Period",
        "UNIT_CONVERSION": "Multiplier",
        "UNIT_MULTIPLIER": "Multiplier",
        "CONVERTCUR": "Rate",
        "DESEASONALIZE": "Seasonal Period"
    }
    numeric_defaults = {"POWER": 2, "ROLLSUM": 3, "ROUND": 2, "%CHANGE": 1, "%EXPGROWTH": 1, "CAGR": 1, "DIFF": 1,
                        "UNIT_CONVERSION": 1.0, "UNIT_MULTIPLIER": 1.0, "CONVERTCUR": 1.0, "DESEASONALIZE": 12}
    show_method = f in ["CROSSSEC", "AGGREGATE", "DISAGGREGATE", "ACCUMULATE", "DESEASONALIZE"]
    method_choices = []
    if f == "CROSSSEC":
        method_choices = ["mean", "sum", "max", "min", "median", "std"]
    elif f == "AGGREGATE":
        method_choices = ["sum", "mean", "max", "min", "first", "last"]
    elif f == "DISAGGREGATE":
        method_choices = ["linear", "spline", "ffill"]
    elif f == "ACCUMULATE":
        method_choices = ["sum", "avg"]
    elif f == "DESEASONALIZE":
        method_choices = ["multiplicative", "additive"]

    show_frequency = f in ["AGGREGATE", "DISAGGREGATE"]
    show_result_type = f == "DIVIDE"
    show_time_col = f in ["AGGREGATE", "DISAGGREGATE", "DESEASONALIZE", "ROLLSUM", "%CHANGE", "%EXPGROWTH", "CAGR", "DIFF", "ACCUMULATE"]
    show_align_end = f in ["AGGREGATE", "DISAGGREGATE"]
    numeric_label = numeric_label_map.get(f, "Parameter")
    numeric_default = numeric_defaults.get(f, None)
    return {
        "f": f, "show_operand": show_operand, "show_numeric": show_numeric,
        "numeric_label": numeric_label, "numeric_default": numeric_default,
        "show_method": show_method, "method_choices": method_choices,
        "show_frequency": show_frequency, "show_result_type": show_result_type,
        "show_time_col": show_time_col, "show_align_end": show_align_end
    }

def update_functions(category):
    category_map = {
        "Basic Arithmetic": ["ADD", "SUBTRACT", "MULTIPLY", "DIVIDE"],
        "Statistical": ["SUM", "CROSSSEC", "ROLLSUM"],
        "Transformation": ["ACCUMULATE", "ANTILOG", "EXP", "LOG", "LOG10", "POWER", "RECIP", "ROUND", "SQRT"],
        "Time Series": ["%CHANGE", "%EXPGROWTH", "CAGR", "DIFF", "AGGREGATE", "DISAGGREGATE", "DESEASONALIZE"],
        "Conversion": ["UNIT_CONVERSION", "UNIT_MULTIPLIER", "CONVERTCUR"]
    }
    functions = category_map.get(category, ["ADD"])
    return gr.update(choices=functions, value=functions[0])

def on_math_function_change(func_name, operand_source, ds_use_in_math):
    global current_filtered_df
    cfg = math_function_config(func_name)
    f = cfg["f"]

    descriptions = {
        "ACCUMULATE": "Running total (sum) or running average over the (time) order.",
        "ADD": "Add a number or column to each value.",
        "SUBTRACT": "Subtract a number or column.",
        "MULTIPLY": "Multiply by a number or column.",
        "DIVIDE": "Divide by a number or column. Output can be float, int, or percent.",
        "ANTILOG": "e^x for each value.",
        "EXP": "Exponential (e^x).",
        "LOG": "Natural log (x>0).",
        "LOG10": "Base-10 log (x>0).",
        "POWER": "Raise each value to a power.",
        "RECIP": "1/x (skips zeros).",
        "ROUND": "Round to N decimals.",
        "SQRT": "Square root (x>=0).",
        "SUM": "Column total (scalar).",
        "ROLLSUM": "Moving window sum (size = Window).",
        "CROSSSEC": "Cross-sectional aggregation: mean/sum/max/min/median/std.",
        "%CHANGE": "Percent change over a lag (ordered by time if provided).",
        "%EXPGROWTH": "Percent change over a lag (same units as %CHANGE).",
        "CAGR": "Compound annual growth rate (%) over given periods.",
        "DIFF": "Difference over lag.",
        "AGGREGATE": "Resample to lower frequency (sum/mean/etc.).",
        "DISAGGREGATE": "Up-sample to higher frequency (linear/spline/ffill).",
        "UNIT_CONVERSION": "Multiply by unit factor.",
        "UNIT_MULTIPLIER": "Multiply by factor.",
        "CONVERTCUR": "Convert currency by rate.",
        "DESEASONALIZE": "Remove seasonal pattern (multiplicative or additive)."
    }
    desc_update = gr.update(value=descriptions.get(f, "Select a function to see details."), visible=True)

    param_help = ""
    if cfg["show_numeric"]:
        if f == "POWER": param_help = "Exponent (e.g., 2 → square)."
        elif f == "ROLLSUM": param_help = "Window length in periods."
        elif f == "ROUND": param_help = "Decimals."
        elif f in ["%CHANGE", "%EXPGROWTH", "DIFF"]: param_help = "Lag period."
        elif f == "CAGR": param_help = "Number of periods between first and last values."
        elif f in ["UNIT_CONVERSION", "UNIT_MULTIPLIER", "CONVERTCUR"]: param_help = "Multiplier/Rate."
        elif f == "DESEASONALIZE": param_help = "Seasonal period (e.g., 12 for monthly)."
    else:
        param_help = "No numeric parameter required."

    if cfg["show_method"]:
        if f == "ACCUMULATE": param_help += " | Method: sum or avg."
        elif f == "CROSSSEC": param_help += " | Method: mean/sum/max/min/median/std."
        elif f == "AGGREGATE": param_help += " | Method: sum/mean/max/min/first/last."
        elif f == "DISAGGREGATE": param_help += " | Method: linear/spline/ffill."
        elif f == "DESEASONALIZE": param_help += " | Model: multiplicative/additive."
    if cfg["show_time_col"]:
        param_help += " | Tip: Choose a Time Column to ensure correct ordering."
    if ds_use_in_math:
        param_help += " | Using deseasonalized input requires a Time Column."

    time_choices = detect_datetime_cols(current_filtered_df)
    show_time = cfg["show_time_col"] or bool(ds_use_in_math)
    return (
        gr.update(visible=cfg["show_operand"], value=operand_source if cfg["show_operand"] else None),
        gr.update(visible=cfg["show_operand"] and (operand_source == "value")),
        gr.update(visible=cfg["show_operand"] and (operand_source == "column")),
        gr.update(visible=cfg["show_numeric"], label=f"{cfg['numeric_label']} (required)" if cfg["show_numeric"] else "Parameter", value=cfg["numeric_default"], interactive=True),
        gr.update(visible=cfg["show_method"], choices=cfg["method_choices"], value=cfg["method_choices"][0] if cfg["method_choices"] else None),
        gr.update(visible=cfg["show_frequency"], value="M" if f == "AGGREGATE" else "D"),
        gr.update(visible=cfg["show_result_type"], value="float"),
        gr.update(visible=show_time, choices=time_choices, value=(time_choices[0] if (show_time and time_choices) else None)),
        gr.update(visible=cfg["show_align_end"], value=True),
        desc_update,
        gr.update(value=f"Tip: {param_help}", visible=True)
    )

def on_operand_source_change(operand_source, func_name):
    cfg = math_function_config(func_name)
    return (gr.update(visible=cfg["show_operand"] and (operand_source == "value")),
            gr.update(visible=cfg["show_operand"] and (operand_source == "column")))

def on_arima_use_ds_change(use):
    return gr.update(visible=bool(use))

def on_math_ds_toggle(enable, func_name):
    global current_filtered_df
    cfg = math_function_config(func_name)
    time_choices = detect_datetime_cols(current_filtered_df)
    # Show DS params when enabled; also show time column if either function requires it or DS is enabled
    show_time = cfg["show_time_col"] or bool(enable)
    return (
        gr.update(visible=bool(enable), value=12),                    # ds_math_period
        gr.update(visible=bool(enable), value="multiplicative"),      # ds_math_model
        gr.update(visible=show_time, choices=time_choices, value=(time_choices[0] if (show_time and time_choices) else None))  # time_column
    )

def toggle_preview():
    global preview_visible
    preview_visible = not preview_visible
    return gr.update(visible=preview_visible), gr.update(value="Hide Preview" if preview_visible else "Show Preview")

def toggle_filtered():
    global filtered_visible
    filtered_visible = not filtered_visible
    return gr.update(visible=filtered_visible), gr.update(value="Hide Filtered Data" if filtered_visible else "Show Filtered Data")

def update_table_preview(table_name):
    global tables_data
    if table_name in tables_data and not tables_data[table_name].empty:
        return tables_data[table_name], gr.update(visible=False)
    return pd.DataFrame(), gr.update(visible=False)

def get_table_preview_only(table_name):
    global tables_data
    return tables_data.get(table_name, pd.DataFrame())

def show_arima_description(show):
    return gr.update(visible=show)

def _initial_math_updates(numeric_cols, all_cols):
    return (
        gr.update(choices=["Basic Arithmetic", "Statistical", "Transformation", "Time Series", "Conversion"], value="Basic Arithmetic"),
        gr.update(choices=["ADD", "SUBTRACT", "MULTIPLY", "DIVIDE"], value="ADD"),
        gr.update(choices=numeric_cols, value=numeric_cols[0] if numeric_cols else None),
        gr.update(visible=True, value="value"),
        gr.update(visible=True, value=None),
        gr.update(choices=numeric_cols, visible=False, value=None),
        gr.update(visible=False, value=None, label="Parameter", interactive=True),
        gr.update(visible=False, choices=[], value=None),
        gr.update(visible=False, value="M"),
        gr.update(visible=False, value="float"),
        gr.update(choices=[], visible=False, value=None),
        gr.update(visible=False, value=True),
        # New DS controls defaults
        gr.update(value=False, visible=True),      # ds_math_use
        gr.update(visible=False, value=12),        # ds_math_period
        gr.update(visible=False, value="multiplicative"),  # ds_math_model
        # Descriptions
        gr.update(value="", visible=True),
        gr.update(value="Tip: Select a function. Fields will appear only when needed.", visible=True)
    )

def _arima_ui_updates_for_df(df):
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    time_cols = detect_datetime_cols(df)
    default_time = time_cols[0] if time_cols else None
    return (
        gr.update(choices=numeric_cols, value=numeric_cols[0] if numeric_cols else None),
        gr.update(choices=time_cols, value=default_time),
        gr.update(value="infer")
    )

def build_filter_updates_from_df(df):
    global filter_columns_map
    updates = []
    filter_columns_map = {}
    cols = df.columns.tolist()
    for idx, col in enumerate(cols[:MAX_FILTERS]):
        filter_columns_map[idx] = col
        unique_vals = df[col].dropna().astype(str).unique().tolist()
        updates.append(visible_filter_options(col, unique_vals))
    for _ in range(MAX_FILTERS - len(cols[:MAX_FILTERS])):
        updates.append(gr.update(visible=False, choices=[], value=None))
    return updates

def update_all_filters(*selections):
    global original_df, filter_columns_map, MAX_UNIQUE_FILTER, active_filter_values
    if original_df.empty:
        return tuple(gr.update(visible=False, choices=[], value=None) for _ in range(MAX_FILTERS))

    df = original_df.copy()
    norm_selections = {}
    active_filter_values = {}
    for i, sel in enumerate(selections):
        col = filter_columns_map.get(i)
        if not col or col not in df.columns:
            continue
        if isinstance(sel, (list, tuple, set, np.ndarray, pd.Series)):
            sel = next((str(x) for x in sel if str(x) != FILTER_ALL_TOKEN and str(x).strip() != ""), None)
        sel = None if sel is None or sel == "" else str(sel)
        norm_selections[i] = sel if sel else FILTER_ALL_TOKEN
        active_filter_values[col] = [sel] if sel else [FILTER_ALL_TOKEN]

    outs = []
    for i in range(MAX_FILTERS):
        col_i = filter_columns_map.get(i)
        if not col_i or col_i not in df.columns:
            outs.append(gr.update(visible=False, choices=[], value=None))
            continue

        ctx = df
        for j in range(i):
            col_j = filter_columns_map.get(j)
            val_j = norm_selections.get(j, FILTER_ALL_TOKEN)
            if col_j and col_j in ctx.columns and val_j != FILTER_ALL_TOKEN:
                ctx = ctx[ctx[col_j].astype(str) == val_j]
                if ctx.empty:
                    break

        unique_vals = [] if ctx.empty else ctx[col_i].dropna().astype(str).unique().tolist()
        choices = [FILTER_ALL_TOKEN] + sorted(unique_vals)
        cur_sel = norm_selections.get(i, FILTER_ALL_TOKEN)
        new_value = cur_sel if cur_sel in choices else FILTER_ALL_TOKEN
        visible = (len(unique_vals)) <= MAX_UNIQUE_FILTER
        outs.append(gr.update(visible=visible, choices=choices, value=new_value, label=str(col_i), interactive=True))

    return tuple(outs)

# ---------------------------
# Callback Functions
# ---------------------------
def load_file(file):
    global original_df, current_filtered_df, preview_visible, filtered_visible, tables_data, active_filter_values
    outputs = []
    if file is None:
        outputs.append(None)  # preview
        outputs.extend([gr.update(visible=False, choices=[], value=None) for _ in range(MAX_FILTERS)])
        outputs.append(None)  # filtered_df
        outputs.append(gr.update(value="Show Preview"))  # toggle_preview_btn
        outputs.append(pd.DataFrame())  # stats_df
        outputs.append(gr.update(choices=[], value=None))  # arima_column
        outputs.append(gr.update(choices=[], value=None))  # arima_time_col
        outputs.append(gr.update(value="infer"))           # arima_freq
        outputs.append(gr.update(choices=["Basic Arithmetic", "Statistical", "Transformation", "Time Series", "Conversion"], value="Basic Arithmetic"))
        outputs.append(gr.update(choices=["ADD", "SUBTRACT", "MULTIPLY", "DIVIDE"], value="ADD"))
        outputs.append(gr.update(choices=[], value=None))  # math_column
        outputs.append(gr.update(visible=False, value="value"))  # operand_source
        outputs.append(gr.update(visible=False, value=None))     # operand_value
        outputs.append(gr.update(choices=[], visible=False, value=None))  # operand_column
        outputs.append(gr.update(visible=False, value=None, label="Parameter", interactive=True))  # numeric_param hidden
        outputs.append(gr.update(visible=False, choices=[], value=None))         # method_param
        outputs.append(gr.update(visible=False, value="M"))                      # frequency_param
        outputs.append(gr.update(visible=False, value="float"))                  # result_type
        outputs.append(gr.update(choices=[], visible=False, value=None))         # time_column
        outputs.append(gr.update(visible=False, value=True))                     # align_end
        # New DS controls
        outputs.append(gr.update(value=False, visible=True))                     # ds_math_use
        outputs.append(gr.update(visible=False, value=12))                       # ds_math_period
        outputs.append(gr.update(visible=False, value="multiplicative"))         # ds_math_model
        outputs.append(gr.update(value="", visible=True))                        # math_function_description
        outputs.append(gr.update(value="Tip: Select a function. Fields will appear only when needed.", visible=True))  # param_helper
        outputs.append(gr.update(choices=[], value=None))  # table_selector
        outputs.append(pd.DataFrame())                     # table_preview
        outputs.append(gr.update(visible=False))             # modified_csv_download
        outputs.append(gr.update(value="Show Filtered Data"))  # toggle_filtered_btn
        outputs.append(gr.update(visible=False))  # current_data_download
        outputs.append(gr.update(visible=False))  # math_result_download
        outputs.append(gr.update(visible=False))  # forecast_download
        outputs.append(gr.update(visible=False))  # ds_series_download
        return tuple(outputs)

    try:
        df = safe_load_df(file)
        original_df = current_filtered_df = df.copy()
        tables_data = {"Original Table": df.copy()}  # Changed from "Table 1" to "Original Table"
        active_filter_values = {}
        preview_visible = False
        filtered_visible = False

        outputs.append(df)  # preview
        outputs.extend(build_filter_updates_from_df(df))

        outputs.append(df)  # filtered_df
        outputs.append(gr.update(value="Show Preview"))  # toggle_preview_btn
        outputs.append(get_statistics(df))  # stats_df

        arima_updates = _arima_ui_updates_for_df(df)
        outputs.extend(arima_updates)  # arima_column, arima_time_col, arima_freq

        # Math defaults
        numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
        math_updates = _initial_math_updates(numeric_cols, df.columns.tolist())
        outputs.extend(math_updates)  # includes ds controls now

        table_selector_update = gr.update(choices=list(tables_data.keys()), value=list(tables_data.keys())[0] if tables_data else None)
        outputs.append(table_selector_update)
        outputs.append(df)  # table_preview
        outputs.append(gr.update(visible=False))  # modified_csv_download

        outputs.append(gr.update(value="Show Filtered Data"))

        # current data download auto
        meta = get_filter_suffix_for_filename()
        cur_df = maybe_attach_static_cols(current_filtered_df, meta)
        cur_path = make_temp_csv(cur_df, "current_data", meta)
        outputs.append(gr.update(value=cur_path, visible=True))  # current_data_download
        outputs.append(gr.update(visible=False))  # math_result_download
        outputs.append(gr.update(visible=False))  # forecast_download
        outputs.append(gr.update(visible=False))  # ds_series_download

        return tuple(outputs)
    except Exception as e:
        print(f"Error loading file: {e}")
        outputs = [None]
        outputs.extend([gr.update(visible=False, choices=[], value=None) for _ in range(MAX_FILTERS)])
        outputs.append(None)
        outputs.append(gr.update(value="Show Preview"))
        outputs.append(pd.DataFrame())
        outputs.append(gr.update(choices=[], value=None))  # arima_column
        outputs.append(gr.update(choices=[], value=None))  # arima_time_col
        outputs.append(gr.update(value="infer"))           # arima_freq
        outputs.append(gr.update(choices=["Basic Arithmetic", "Statistical", "Transformation", "Time Series", "Conversion"], value="Basic Arithmetic"))
        outputs.append(gr.update(choices=["ADD", "SUBTRACT", "MULTIPLY", "DIVIDE"], value="ADD"))
        outputs.append(gr.update(choices=[], value=None))
        outputs.append(gr.update(visible=False, value="value"))
        outputs.append(gr.update(visible=False, value=None))
        outputs.append(gr.update(choices=[], visible=False, value=None))
        outputs.append(gr.update(visible=False, value=None, label="Parameter", interactive=True))
        outputs.append(gr.update(visible=False, choices=[], value=None))
        outputs.append(gr.update(visible=False, value="M"))
        outputs.append(gr.update(visible=False, value="float"))
        outputs.append(gr.update(choices=[], visible=False, value=None))
        outputs.append(gr.update(visible=False, value=True))
        # DS controls defaults on error
        outputs.append(gr.update(value=False, visible=True))      # ds_math_use
        outputs.append(gr.update(visible=False, value=12))        # ds_math_period
        outputs.append(gr.update(visible=False, value="multiplicative"))  # ds_math_model
        outputs.append(gr.update(value="", visible=True))
        outputs.append(gr.update(value="Tip: Select a function. Fields will appear only when needed.", visible=True))
        outputs.append(gr.update(choices=[], value=None))
        outputs.append(pd.DataFrame())
        outputs.append(gr.update(visible=False))
        outputs.append(gr.update(value="Show Filtered Data"))
        outputs.append(gr.update(visible=False))  # current_data_download
        outputs.append(gr.update(visible=False))  # math_result_download
        outputs.append(gr.update(visible=False))  # forecast_download
        outputs.append(gr.update(visible=False))  # ds_series_download
        return tuple(outputs)

def apply_filters(*filters_values):
    global original_df, current_filtered_df, tables_data, active_filter_values

    df = original_df.copy()
    active_filter_values = {}

    for i, selected in enumerate(filters_values):
        col_name = filter_columns_map.get(i, None)
        if not col_name or col_name not in df.columns:
            continue

        if isinstance(selected, (list, tuple, set, np.ndarray, pd.Series)):
            selected = next((str(x) for x in selected if str(x) != FILTER_ALL_TOKEN and str(x).strip() != ""), None)
        elif selected is not None:
            selected = str(selected)

        if selected is None or selected == "" or selected == FILTER_ALL_TOKEN:
            active_filter_values[col_name] = [FILTER_ALL_TOKEN]
            continue

        col_str = df[col_name].astype(str)
        if selected in set(col_str.dropna().unique().tolist()):
            df = df[col_str == selected]
            active_filter_values[col_name] = [selected]
        else:
            active_filter_values[col_name] = [FILTER_ALL_TOKEN]

    df = coerce_datetime_columns(df)
    current_filtered_df = df.copy()
    tables_data["Filtered Table"] = df.copy()

    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    time_cols = detect_datetime_cols(df)
    default_time = time_cols[0] if time_cols else None

    math_updates = _initial_math_updates(numeric_cols, df.columns.tolist())
    meta = get_filter_suffix_for_filename()
    cur_df = maybe_attach_static_cols(current_filtered_df, meta)
    cur_path = make_temp_csv(cur_df, "current_data", meta)

    return (
        gr.update(value=df, visible=True),              # filtered_df
        gr.update(value="Hide Filtered Data"),          # toggle_filtered_btn
        get_statistics(df),                             # stats_df
        gr.update(choices=numeric_cols, value=numeric_cols[0] if numeric_cols else None),  # arima_column
        gr.update(choices=time_cols, value=default_time),  # arima_time_col
        gr.update(value="infer"),  # arima_freq
        math_updates[0],  # math_category
        math_updates[1],  # math_function
        math_updates[2],  # math_column
        math_updates[3],  # operand_source
        math_updates[4],  # operand_value
        math_updates[5],  # operand_column
        math_updates[6],  # numeric_param
        math_updates[7],  # method_param
        math_updates[8],  # frequency_param
        math_updates[9],  # result_type
        gr.update(choices=time_cols, visible=False, value=None), # time_column
        math_updates[11], # align_end
        math_updates[12], # ds_math_use
        math_updates[13], # ds_math_period
        math_updates[14], # ds_math_model
        gr.update(choices=list(tables_data.keys()), value="Filtered Table"),
        math_updates[15], # math_function_description
        math_updates[16],  # param_helper
        gr.update(value=cur_path, visible=True)  # current_data_download
    )

def generate_download_file():
    global current_filtered_df
    if current_filtered_df.empty:
        raise gr.Error("No data available to download!")
    meta = get_filter_suffix_for_filename()
    path = make_temp_csv(maybe_attach_static_cols(current_filtered_df, meta), "current_data_manual", meta)
    return gr.update(value=path, visible=True)

def clear_all_filters():
    global original_df, current_filtered_df, preview_visible, filtered_visible, tables_data, active_filter_values
    if original_df.empty:
        outs = [None]
        outs.extend([gr.update(value=None) for _ in range(MAX_FILTERS)])  # filters
        outs.append(None)  # preview
        outs.append(gr.update(value="Show Preview"))
        outs.append(pd.DataFrame())  # stats
        outs.append(gr.update(choices=[], value=None))  # arima_column
        outs.append(gr.update(choices=[], value=None))  # arima_time_col
        outs.append(gr.update(value="infer"))           # arima_freq
        outs.append(gr.update(choices=["Basic Arithmetic", "Statistical", "Transformation", "Time Series", "Conversion"], value="Basic Arithmetic"))
        outs.append(gr.update(choices=["ADD", "SUBTRACT", "MULTIPLY", "DIVIDE"], value="ADD"))
        outs.append(gr.update(choices=[], value=None))  # math_column
        outs.append(gr.update(visible=False, value="value"))  # operand_source
        outs.append(gr.update(visible=False, value=None))     # operand_value
        outs.append(gr.update(choices=[], visible=False, value=None))  # operand_column
        outs.append(gr.update(visible=False, value=None, label="Parameter", interactive=True))  # numeric_param hidden
        outs.append(gr.update(visible=False, choices=[], value=None))         # method_param
        outs.append(gr.update(visible=False, value="M"))                      # frequency_param
        outs.append(gr.update(visible=False, value="float"))                  # result_type
        outs.append(gr.update(choices=[], visible=False, value=None))         # time_column
        outs.append(gr.update(visible=False, value=True))                     # align_end
        # DS controls
        outs.append(gr.update(value=False, visible=True))      # ds_math_use
        outs.append(gr.update(visible=False, value=12))        # ds_math_period
        outs.append(gr.update(visible=False, value="multiplicative"))  # ds_math_model
        outs.append(gr.update(value="", visible=True))                        # math_function_description
        outs.append(gr.update(value="Tip: Select a function. Fields will appear only when needed.", visible=True))  # param_helper
        outs.append(gr.update(choices=[], value=None))  # table_selector
        outs.append(pd.DataFrame())  # table_preview
        outs.append(gr.update(visible=False))  # modified_csv_download
        outs.append(gr.update(value="Show Filtered Data"))
        outs.append(gr.update(visible=False))  # current_data_download
        return tuple(outs)

    current_filtered_df = coerce_datetime_columns(original_df.copy())
    tables_data = {"Original Table": original_df.copy()}  # Changed from "Table 1" to "Original Table"
    active_filter_values = {}
    numeric_cols = current_filtered_df.select_dtypes(include=[np.number]).columns.tolist()
    time_cols = detect_datetime_cols(current_filtered_df)
    default_time = time_cols[0] if time_cols else None

    preview_visible = False
    filtered_visible = False

    meta = get_filter_suffix_for_filename()
    cur_df = maybe_attach_static_cols(current_filtered_df, meta)
    cur_path = make_temp_csv(cur_df, "current_data", meta)

    outs = [current_filtered_df]  # filtered_df
    outs.extend(build_filter_updates_from_df(original_df))  # filters reset to full dataset
    outs.append(current_filtered_df)  # preview
    outs.append(gr.update(value="Show Preview"))
    outs.append(get_statistics(current_filtered_df))
    outs.append(gr.update(choices=numeric_cols, value=numeric_cols[0] if numeric_cols else None))
    outs.append(gr.update(choices=time_cols, value=default_time))
    outs.append(gr.update(value="infer"))

    math_updates = _initial_math_updates(numeric_cols, current_filtered_df.columns.tolist())
    outs.extend(math_updates)

    outs.append(gr.update(choices=list(tables_data.keys()), value=list(tables_data.keys())[0]))  # table_selector
    outs.append(current_filtered_df)  # table_preview
    outs.append(gr.update(visible=False))  # modified_csv_download
    outs.append(gr.update(value="Show Filtered Data"))
    outs.append(gr.update(value=cur_path, visible=True))  # current_data_download
    return tuple(outs)

def get_statistics(df):
    if df.empty:
        return pd.DataFrame()

    df_num = df.copy()
    for c in df_num.columns:
        if df_num[c].dtype == object:
            coerced = robust_to_numeric(df_num[c])
            if coerced.notna().mean() >= 0.5:
                df_num[c] = coerced

    numeric_df = df_num.select_dtypes(include=[np.number])
    meaningful_cols = [
        col for col in numeric_df.columns
        if col.lower() not in [x.lower() for x in EXCLUDED_STAT_COLS] and numeric_df[col].nunique() > 1
    ]
    stats = []
    for col in meaningful_cols:
        col_s = numeric_df[col].dropna()
        if col_s.empty:
            continue
        stats.append({
            "Column": col,
            "Min": round(col_s.min(), 4),
            "Max": round(col_s.max(), 4),
            "Average": round(col_s.mean(), 4),
            "Std Dev": round(col_s.std(ddof=1), 4) if len(col_s) > 1 else 0.0,
            "Skewness": round(col_s.skew(), 4) if len(col_s) > 2 else 0.0
        })
    return pd.DataFrame(stats)

# --- Enhanced CSS Styling ---
css = """
/* General Styles */
body {
    font-family: 'Inter', -apple-system, BlinkMacSystemFont, sans-serif;
    background: linear-gradient(135deg, #f0f4f8 0%, #d9e2ec 100%);
    color: #334155;
}

/* Main Title */
.main-title {
    text-align: center;
    background: linear-gradient(90deg, #2563eb, #3b82f6);
    -webkit-background-clip: text;
    -webkit-text-fill-color: transparent;
    font-size: 2.8em;
    font-weight: 800;
    margin-bottom: 0.3em;
    padding-top: 1em;
    text-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}
.subtitle {
    text-align: center;
    color: #475569;
    font-size: 1.3em;
    font-weight: 500;
    margin-bottom: 2em;
}

/* Sections */
.upload-section, .filter-section, .table-manip-section, .math-panel, .forecast-section {
    background: white;
    border-radius: 16px;
    padding: 24px;
    margin-bottom: 24px;
    box-shadow: 0 6px 16px rgba(0, 0, 0, 0.08);
    transition: transform 0.3s ease, box-shadow 0.3s ease;
}
.upload-section { border: 2px solid #3b82f6; background: #eff6ff; }
.filter-section { border: 2px solid #22c55e; background: #f0fdf4; }
.table-manip-section { border: 2px solid #eab308; background: #fefce8; }
.math-panel { border: 2px solid #8b5cf6; background: #f5f3ff; }
.forecast-section { border: 2px solid #f97316; background: #fff7ed; }
.section:hover { transform: translateY(-4px); box-shadow: 0 8px 20px rgba(0, 0, 0, 0.12); }

/* Buttons */
.button {
    border-radius: 8px;
    padding: 12px 24px;
    font-weight: 600;
    transition: all 0.3s ease;
    margin: 6px;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
}
.button:hover { transform: translateY(-2px); box-shadow: 0 4px 8px rgba(0, 0, 0, 0.15); }
.download-btn { background: linear-gradient(90deg, #22c55e, #4ade80); color: white; }
.clear-btn { background: linear-gradient(90deg, #ef4444, #f87171); color: white; }
.apply-btn { background: linear-gradient(90deg, #3b82f6, #60a5fa); color: white; }
.toggle-btn { background: linear-gradient(90deg, #6b7280, #9ca3af); color: white; }
.save-btn { background: linear-gradient(90deg, #f97316, #fb923c); color: white; }
.math-btn { background: linear-gradient(90deg, #8b5cf6, #a78bfa); color: white; }
.forecast-btn { background: linear-gradient(90deg, #f97316, #fb923c); color: white; }

/* Dataframes */
.gradio-dataframe table {
    border-collapse: separate;
    border-spacing: 0 8px;
}
.gradio-dataframe th, .gradio-dataframe td {
    background: #f9fafb;
    border: none;
    padding: 12px;
    border-radius: 6px;
}
.gradio-dataframe th {
    background: #e5e7eb;
    font-weight: 600;
}

/* Dropdowns and Inputs */
.gradio-dropdown, .gradio-number, .gradio-radio, .gradio-checkbox {
    border-radius: 8px;
    border: 1px solid #d1d5db;
    box-shadow: 0 1px 2px rgba(0, 0, 0, 0.05);
}
.gradio-dropdown:hover, .gradio-number:hover {
    border-color: #9ca3af;
}

/* Markdown */
.gradio-markdown {
    font-size: 1.05em;
    line-height: 1.6;
}

/* Tabs */
.gradio-tabs .tabitem {
    border-radius: 12px 12px 0 0;
    background: #f3f4f6;
    padding: 12px 20px;
    font-weight: 600;
}
.gradio-tabs .tabitem.selected {
    background: white;
    border-bottom: none;
    box-shadow: 0 2px 4px rgba(0, 0, 0, 0.05);
}
"""

# --- Gradio UI ---
with gr.Blocks(css=css, theme=gr.themes.Soft(primary_hue="blue", secondary_hue="green", font=[gr.themes.GoogleFont('Inter'), 'system-ui'])) as demo:
    gr.Markdown(
        """
        <h1 class='main-title'>🧮 DataFlow Analytics Pro</h1>
        <p class='subtitle'>Advanced Data Transformation, Filtering, and Forecasting Platform</p>
        """,
        elem_id="main-title"
    )

    with gr.Tabs():
        with gr.Tab("Upload & Filter"):
            with gr.Accordion("📂 Upload Data", open=True, elem_classes=["upload-section"]):
                file_input = gr.File(label="Upload CSV or Excel File", file_types=[".csv", ".xlsx", ".xls"], elem_id="file-upload")
                toggle_preview_btn = gr.Button("🔍 Preview Data", elem_classes=["toggle-btn"])
                preview = gr.Dataframe(label="Data Preview", interactive=False, visible=False)

            with gr.Accordion("🗂️ Apply Filters", open=True, elem_classes=["filter-section"]):
                gr.Markdown("Select filters progressively from left to right.")
                with gr.Row(variant="compact"):
                    filters = [
                        gr.Dropdown(multiselect=False, visible=False, label=f"Filter {i+1}", allow_custom_value=False, elem_id=f"filter-{i}")
                        for i in range(MAX_FILTERS)
                    ]

            with gr.Row(variant="compact"):
                apply_button = gr.Button("✅ Apply Filters", elem_classes=["apply-btn"])
                download_btn = gr.Button("📥 Snapshot Download", elem_classes=["download-btn"])
                clear_btn = gr.Button("🧹 Clear All", elem_classes=["clear-btn"])

            current_data_download = gr.File(label="⬇️ Current Data (Auto-Updated)", visible=False)

            toggle_filtered_btn = gr.Button("📝 View Filtered Data", elem_classes=["toggle-btn"])
            filtered_df = gr.Dataframe(label="Filtered Data (Editable)", interactive=True, visible=False)
            stats_df = gr.Dataframe(label="📊 Key Statistics", interactive=False)

            stats_desc = gr.Markdown(
                """
                **Std Dev**: Measures data spread.
                **Skewness**: Indicates distribution asymmetry (positive: right-skewed, negative: left-skewed).
                """,
                visible=True
            )

        with gr.Tab("Math Operations"):
            with gr.Accordion("🔢 Perform Calculations", open=True, elem_classes=["math-panel"]):
                gr.Markdown("Choose category and function to transform your data.")
                with gr.Row(variant="compact"):
                    math_category = gr.Dropdown(
                        label="Category",
                        choices=["Basic Arithmetic", "Statistical", "Transformation", "Time Series", "Conversion"],
                        value="Basic Arithmetic",
                        interactive=True,
                        elem_id="math-category"
                    )
                    math_function = gr.Dropdown(
                        label="Function",
                        choices=["ADD", "SUBTRACT", "MULTIPLY", "DIVIDE"],
                        value="ADD",
                        interactive=True,
                        elem_id="math-function"
                    )
                    math_column = gr.Dropdown(label="Column", choices=[], interactive=True, elem_id="math-column")

                math_function_description = gr.Markdown("", visible=True)

                with gr.Row(variant="compact"):
                    operand_source = gr.Radio(label="Operand From", choices=["value", "column"], value="value", visible=True)
                    operand_value = gr.Number(label="Numeric Value", visible=True)
                    operand_column = gr.Dropdown(label="From Column", choices=[], visible=False)

                    numeric_param = gr.Number(label="Parameter", visible=False, interactive=True)
                    method_param = gr.Dropdown(label="Method", choices=[], visible=False)
                    frequency_param = gr.Dropdown(label="Frequency", choices=["D", "W", "M", "Q", "Y"], value="M", visible=False)
                    result_type = gr.Radio(label="Output Type", choices=["float", "int", "percent"], value="float", visible=False)
                    time_column = gr.Dropdown(label="Time Column", choices=[], visible=False)
                    align_end = gr.Checkbox(label="End-Aligned Periods", value=True, visible=False)

                with gr.Row(variant="compact"):
                    ds_math_use = gr.Checkbox(label="Deseasonalize Input", value=False)
                    ds_math_period = gr.Number(label="Seasonal Period", value=12, visible=False)
                    ds_math_model = gr.Radio(label="Model", choices=["multiplicative", "additive"], value="multiplicative", visible=False)

                with gr.Row(variant="compact"):
                    add_to_data_chk = gr.Checkbox(label="Add to Dataset", value=False)
                    result_col_name = gr.Textbox(label="New Column Name", placeholder="e.g., Result_Column")
                    show_original_chk = gr.Checkbox(label="Overlay Original", value=True)

                param_helper = gr.Markdown("Tip: Select a function. Fields will appear only when needed.", visible=True)

                math_btn = gr.Button("▶️ Apply Function", elem_classes=["math-btn"])
                math_output = gr.Dataframe(label="Results Table", interactive=False)
                math_status = gr.Textbox(label="Operation Status", interactive=False)
                math_result_download = gr.File(label="⬇️ Download Results", visible=False)
                math_visualization = gr.Plot(label="Visualization")

        with gr.Tab("ARIMA Forecasting"):
            with gr.Accordion("🔮 Generate Forecasts", open=True, elem_classes=["forecast-section"]):
                gr.Markdown("ARIMA model with optional deseasonalization for better accuracy.")
                with gr.Row(variant="compact"):
                    arima_column = gr.Dropdown(label="Forecast Column", choices=[], interactive=True)
                    arima_time_col = gr.Dropdown(label="Time Column", choices=[], interactive=True)
                    arima_freq = gr.Dropdown(label="Frequency", choices=["infer", "D", "W", "M", "Q", "Y"], value="infer", interactive=True)
                with gr.Row(variant="compact"):
                    arima_steps = gr.Number(label="Steps Ahead", value=5, interactive=True)
                    arima_p = gr.Number(label="AR (p)", value=1, interactive=True)
                    arima_d = gr.Number(label="Diff (d)", value=1, interactive=True)
                    arima_q = gr.Number(label="MA (q)", value=1, interactive=True)
                    arima_btn = gr.Button("🚀 Run Forecast", elem_classes=["forecast-btn"])

                arima_param_help = gr.Markdown(
                    "**ARIMA Guide**: p (autoregression), d (differencing), q (moving average). Start with low values.",
                    visible=True
                )

                gr.Markdown("#### Deseasonalization Settings")
                with gr.Row(variant="compact"):
                    ds_overlay = gr.Checkbox(label="Show Deseasonalized Overlay", value=True)
                    ds_period = gr.Number(label="Period (e.g., 12)", value=12)
                    ds_model = gr.Radio(label="Type", choices=["multiplicative", "additive"], value="multiplicative")
                with gr.Row(variant="compact"):
                    ds_use_arima = gr.Checkbox(label="Use for ARIMA Input", value=False)
                    ds_reseason_out = gr.Checkbox(label="Re-seasonalize Output", value=True, visible=False)

                with gr.Column():
                    arima_output = gr.Dataframe(label="Forecast Table", interactive=False)
                    arima_plot = gr.Plot(label="Forecast Chart")
                    ds_preview_plot = gr.Plot(label="Deseason Preview")
                    arima_metrics = gr.Textbox(label="Accuracy Metrics", interactive=False)
                    forecast_download = gr.File(label="⬇️ Download Forecast", visible=False)
                    ds_series_download = gr.File(label="⬇️ Download Deseasonalized", visible=False)

        with gr.Tab("Table Editor"):
            with gr.Accordion("✏️ Edit Tables", open=True, elem_classes=["table-manip-section"]):
                gr.Markdown("Select and edit tables interactively.")
                with gr.Row(variant="compact"):
                    table_selector = gr.Dropdown(
                        label="Choose Table",
                        choices=[],
                        interactive=True
                    )
                    update_preview_btn = gr.Button(
                        "🔄 Refresh",
                        elem_classes=["toggle-btn"]
                    )
                table_preview = gr.Dataframe(
                    label="Editable Table",
                    interactive=True,
                    wrap=True,
                    datatype=["str", "number", "bool", "date", "datetime"]
                )
                with gr.Row(variant="compact"):
                    save_changes_btn = gr.Button(
                        "💾 Save Edits",
                        elem_classes=["save-btn"]
                    )
                    modification_status = gr.Textbox(
                        label="Edit Status",
                        interactive=False,
                        value="No changes applied yet"
                    )
                modified_csv_download = gr.File(
                    label="⬇️ Download Edited Table",
                    interactive=False,
                    visible=False
                )

    download_output = gr.File(label="⬇️ Manual Snapshot", visible=False)

    # --- Event Handlers ---

    # File load
    file_input.change(
        fn=load_file,
        inputs=file_input,
        outputs=[
            preview,
            *filters,
            filtered_df,
            toggle_preview_btn,
            stats_df,
            arima_column,
            arima_time_col,
            arima_freq,
            math_category, math_function, math_column,
            operand_source, operand_value, operand_column,
            numeric_param, method_param, frequency_param, result_type, time_column, align_end,
            ds_math_use, ds_math_period, ds_math_model,
            math_function_description, param_helper,
            table_selector, table_preview, modified_csv_download,
            toggle_filtered_btn,
            current_data_download,
            math_result_download,
            forecast_download,
            ds_series_download
        ]
    )

    # Progressive cascading filters (left-to-right)
    for f in filters:
        f.change(
            fn=update_all_filters,
            inputs=filters,
            outputs=filters
        )

    # Apply filters
    apply_button.click(
        fn=apply_filters,
        inputs=filters,
        outputs=[
            filtered_df, toggle_filtered_btn, stats_df,
            arima_column, arima_time_col, arima_freq,
            math_category, math_function, math_column,
            operand_source, operand_value, operand_column,
            numeric_param, method_param, frequency_param, result_type,
            time_column, align_end,
            ds_math_use, ds_math_period, ds_math_model,
            table_selector,
            math_function_description, param_helper,
            current_data_download
        ]
    )

    # Manual download snapshot
    download_btn.click(fn=generate_download_file, outputs=[download_output])

    # Clear filters
    clear_btn.click(
        fn=clear_all_filters,
        outputs=[
            filtered_df, *filters, preview, toggle_preview_btn, stats_df,
            arima_column, arima_time_col, arima_freq,
            math_category, math_function, math_column,
            operand_source, operand_value, operand_column,
            numeric_param, method_param, frequency_param, result_type, time_column, align_end,
            ds_math_use, ds_math_period, ds_math_model,
            math_function_description, param_helper,
            table_selector, table_preview, modified_csv_download, toggle_filtered_btn,
            current_data_download
        ]
    )

    # Math category change
    math_category.change(
        fn=update_functions,
        inputs=math_category,
        outputs=math_function
    )

    # Math function UI dynamics (now aware of DS toggle)
    math_function.change(
        fn=on_math_function_change,
        inputs=[math_function, operand_source, ds_math_use],
        outputs=[
            operand_source, operand_value, operand_column,
            numeric_param, method_param, frequency_param, result_type, time_column, align_end,
            math_function_description, param_helper
        ]
    )
    operand_source.change(
        fn=on_operand_source_change,
        inputs=[operand_source, math_function],
        outputs=[operand_value, operand_column]
    )

    # Toggle: DS for Math
    ds_math_use.change(
        fn=on_math_ds_toggle,
        inputs=[ds_math_use, math_function],
        outputs=[ds_math_period, ds_math_model, time_column]
    )

    # ARIMA DS reseason visibility
    ds_use_arima.change(
        fn=on_arima_use_ds_change,
        inputs=ds_use_arima,
        outputs=ds_reseason_out
    )

    # Apply math function
    math_btn.click(
        fn=apply_math_function,
        inputs=[math_function, math_column, operand_source, operand_value, operand_column,
                numeric_param, method_param, frequency_param, result_type, time_column, align_end,
                ds_math_use, ds_math_period, ds_math_model,
                add_to_data_chk, result_col_name, show_original_chk],
        outputs=[
            math_output, math_status, math_result_download, current_data_download, math_visualization,
            arima_column, arima_time_col, math_column, table_selector, table_preview
        ]
    )

    # ARIMA + Deseasonalization (updated)
    arima_btn.click(
        fn=run_arima,
        inputs=[arima_column, arima_steps, arima_p, arima_d, arima_q, arima_time_col, arima_freq,
                ds_overlay, ds_period, ds_model, ds_use_arima, ds_reseason_out],
        outputs=[arima_output, arima_plot, arima_metrics, ds_preview_plot, forecast_download, ds_series_download]
    )

    # Toggle preview/filtered visibility
    toggle_preview_btn.click(
        fn=toggle_preview,
        inputs=None,
        outputs=[preview, toggle_preview_btn]
    )
    toggle_filtered_btn.click(
        fn=toggle_filtered,
        inputs=None,
        outputs=[filtered_df, toggle_filtered_btn]
    )

    # Table manipulation
    table_selector.change(
        fn=update_table_preview,
        inputs=[table_selector],
        outputs=[table_preview, modified_csv_download]
    )
    update_preview_btn.click(
        fn=get_table_preview_only,
        inputs=[table_selector],
        outputs=[table_preview]
    )
    save_changes_btn.click(
        fn=handle_table_modification,
        inputs=[table_selector, table_preview],
        outputs=[modified_csv_download, modification_status, current_data_download]
    )

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://64fb8ef75893c3ef2d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
